# 📝 Kaggle Educator/Tutor Capstone Notebook

## This notebook demonstrates a multi-agent Educator/Tutor system using Google ADK.
## Agents communicate via A2A protocol. Includes custom tool, sessions, memory, and Gemini LLM.
# --------------------------------------------------------------------
## The Pitch

### Problem:
 - Students need instant, personalized tutoring in various subjects. 
 - Educators cannot respond to every query in real time due to resource constraints.

### Solution:
 - Tutor Agent: Main instructor handling user queries.
 - Subject Knowledge Agent: Sub-agent providing domain-specific content.
 - Agents communicate via A2A to deliver real-time, accurate answers.
 - Custom tool: `get_educational_content(topic)` for subject content lookup.
 - Sessions & memory: Tracks user queries and context.

### Value:
 - Personalized tutoring available anytime.
 - Multi-agent collaboration for enhanced accuracy.
 - Easily extendable to new subjects or topics.

## Technical Flow:
 - Student asks a question.
 - Orchestrator decides which agent to invoke.
 - That agent call sub-agents/tools.
 - Final combined response is delivered to the student.

### This demonstrates concepts from the course:

<strong style="color:#0057FF;">A2A interactions</strong>, 
<strong style="color:#0057FF;">Composable agent workflows</strong>, 
<strong style="color:#0057FF;">Function calling and tool use</strong>, 
<strong style="color:#0057FF;">Message routing</strong>, 
<strong style="color:#0057FF;">Structured outputs</strong>, 
<strong style="color:#0057FF;">Multi-Agent Extensions: Parallel Agents</strong>


## Technical Highlights:
  The project uses **Gemini** for: 
  - Agent reasoning
  - Function calling
  - Education-style content generation
  - Structured JSON tool outputs
  - Agent evaluation

## Architecture: Orchestrator-Worker Pattern
                                                     Student Query 
                                                         ⬇️  
                                                     Tutor Agent  
                                                         ⬇️ (A2A call)  
                                                     Knowledge Agent  
                                                         ⬇️  
                                                     Response to Student

## Demonstration Flow: A2A Protocol Delegation
| Step | Actor | Action & Explanation |
|------|--------|----------------------|
| **1. Initial Query** | **User (Student)** | The user sends a subject-specific query (e.g., *“Can you explain Python loops?”*) to the **Tutor Agent (Orchestrator)**. |
| **2. Reasoning & Planning** | **Tutor Agent (Orchestrator)** | The Tutor Agent’s LLM analyzes the request and determines it requires detailed subject-matter knowledge handled by the **Subject Knowledge Agent**. |
| **3. Delegation via A2A** | **Tutor Agent → Subject Knowledge Agent** | Acting as an **A2A Client**, the Tutor Agent delegates the task (e.g., *Retrieve detailed content on “Python loops”*) to the **Subject Knowledge Agent (A2A Server)** using the standardized A2A protocol (using a standardized message/task structure).|
| **4. Task Execution** | **Subject Knowledge Agent (Worker)** | The Subject Knowledge Agent executes its specialized workflow: <br>• Uses custom tool `get_educational_content(topic)` to fetch information <br>• Generates a structured educational response using Gemini |
| **5. Result Handoff via A2A** | **Subject Knowledge Agent → Tutor Agent** | The Subject Knowledge Agent returns the completed, structured content back to the Tutor Agent using the A2A protocol. |
| **6. Final Synthesis** | **Tutor Agent** | The Tutor Agent receives the structured content, synthesizes it with the **session & memory** context, and formats the final, personalized, and engaging educational answer for the User.|

In [47]:
# 1. Setup & Authentication

import os
from kaggle_secrets import UserSecretsClient

# Step 0: Google API Setup

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Setup and authentication complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

In [48]:
# 2. Import Libraries

import json
import requests
import subprocess
import time
import uuid
import warnings

from google.adk.agents import LlmAgent
from google.adk.agents.remote_a2a_agent import (
    RemoteA2aAgent,
    AGENT_CARD_WELL_KNOWN_PATH,
)
from google.adk.a2a.utils.agent_to_a2a import to_a2a
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

warnings.filterwarnings("ignore")
print("✅ ADK components imported successfully.")


In [49]:
# 3. Retry Configuration for LLM

retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

In [50]:
# 4. Custom Tool: Subject Knowledge Lookup

def get_educational_content(topic: str) -> str:
    """
    Simulated educational knowledge base tool.
    Returns subject/topic-specific explanations or examples.
    """
    knowledge_base = {
        "python loops": "Python loops include for-loops and while-loops. Example: for i in range(5): print(i)",
        "newton's laws": "Newton's laws describe motion: 1) Inertia, 2) F=ma, 3) Action-Reaction",
        "photosynthesis": "Photosynthesis is the process where plants convert sunlight, CO2, and water into glucose and oxygen.",
        "calculus derivative": "The derivative represents the rate of change of a function. Example: d/dx(x^2) = 2x",
        "world war ii": "World War II was from 1939-1945, involving major global powers and ending with Allied victory.",
        "linear algebra matrix": "A matrix is a rectangular array of numbers. Example: [[1,2],[3,4]]",
    }

    key = topic.lower().strip()
    if key in knowledge_base:
        return f"Topic: {knowledge_base[key]}"
    else:
        available = ", ".join([t.title() for t in knowledge_base.keys()])
        return f"Sorry, I don't have content for '{topic}'. Available topics: {available}"

In [51]:
# 5. Subject Knowledge Agent (Sub-agent)

knowledge_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="knowledge_agent",
    description="Knowledge base agent that provides educational content across multiple subjects.",
    instruction="""
    You are an educational knowledge specialist.
    When asked about a topic, use the get_educational_content tool to fetch accurate explanations.
    Provide clear, student-friendly answers with examples if possible.
    """,
    tools=[get_educational_content],
)

print("✅ Knowledge Base Agent created successfully!")

In [52]:
# 6. Convert Knowledge Agent to A2A App

knowledge_a2a_app = to_a2a(knowledge_agent, port=8001)
print("✅ Knowledge Base Agent is now A2A-compatible!")

# Save the agent code for uvicorn
knowledge_agent_code = '''
import os
from google.adk.agents import LlmAgent
from google.adk.a2a.utils.agent_to_a2a import to_a2a
from google.adk.models.google_llm import Gemini
from google.genai import types

retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

def get_educational_content(topic: str) -> str:
    knowledge_base = {
        "python loops": "Python loops include for-loops and while-loops. Example: for i in range(5): print(i)",
        "newton's laws": "Newton's laws describe motion: 1) Inertia, 2) F=ma, 3) Action-Reaction",
        "photosynthesis": "Photosynthesis is the process where plants convert sunlight, CO2, and water into glucose and oxygen.",
        "calculus derivative": "The derivative represents the rate of change of a function. Example: d/dx(x^2) = 2x",
        "world war ii": "World War II was from 1939-1945, involving major global powers and ending with Allied victory.",
        "linear algebra matrix": "A matrix is a rectangular array of numbers. Example: [[1,2],[3,4]]",
    }
    key = topic.lower().strip()
    if key in knowledge_base:
        return f"Topic: {knowledge_base[key]}"
    else:
        available = ", ".join([t.title() for t in knowledge_base.keys()])
        return f"Sorry, I don't have content for '{topic}'. Available topics: {available}"

knowledge_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="knowledge_agent",
    description="Knowledge base agent that provides educational content across multiple subjects.",
    instruction="""
    You are an educational knowledge specialist.
    When asked about a topic, use the get_educational_content tool to fetch accurate explanations.
    Provide clear, student-friendly answers with examples if possible.
    """,
    tools=[get_educational_content]
)

app = to_a2a(knowledge_agent, port=8001)
'''

with open("/tmp/knowledge_agent_server.py", "w") as f:
    f.write(knowledge_agent_code)
print("📝 Knowledge Agent code saved to /tmp/knowledge_agent_server.py")

In [53]:
# 7. Start Knowledge Agent Server

server_process = subprocess.Popen(
    [
        "uvicorn",
        "knowledge_agent_server:app",
        "--host", "localhost",
        "--port", "8001",
    ],
    cwd="/tmp",
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    env={**os.environ},
)

print("🚀 Starting Knowledge Base Agent server...")
max_attempts = 30
for attempt in range(max_attempts):
    try:
        response = requests.get("http://localhost:8001/.well-known/agent-card.json", timeout=1)
        if response.status_code == 200:
            print(f"\n✅ Knowledge Agent server is running at http://localhost:8001")
            break
    except requests.exceptions.RequestException:
        time.sleep(5)
        print(".", end="", flush=True)
else:
    print("\n⚠️ Server may not be ready yet. Check manually if needed.")

globals()["knowledge_server_process"] = server_process

# Fetch agent card
try:
    response = requests.get("http://localhost:8001/.well-known/agent-card.json", timeout=5)
    if response.status_code == 200:
        agent_card = response.json()
        print("📋 Knowledge Agent Card:")
        print(json.dumps(agent_card, indent=2))
except requests.exceptions.RequestException as e:
    print(f"❌ Error fetching agent card: {e}")


In [54]:
# 8. Create Remote Proxy Agent

remote_knowledge_agent = RemoteA2aAgent(
    name="knowledge_agent",
    description="Remote knowledge agent providing educational content.",
    agent_card=f"http://localhost:8001{AGENT_CARD_WELL_KNOWN_PATH}",
)

print("✅ Remote Knowledge Agent proxy created!")


In [55]:
# 9. Create Tutor Agent

tutor_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="tutor_agent",
    description="Tutor agent that answers student questions using the Knowledge Agent.",
    instruction="""
    You are a friendly and professional tutor agent.
    
    When students ask about topics:
    1. Use the knowledge_agent sub-agent to fetch educational content.
    2. Provide clear, easy-to-understand answers with examples.
    3. Be patient, helpful, and encouraging.
    """,
    sub_agents=[remote_knowledge_agent],
)

print("✅ Tutor Agent created!")
print("   Ready to help students with questions!")

In [56]:
# 10. Async Test Function

async def test_a2a_communication(user_query: str):
    session_service = InMemorySessionService()
    app_name = "tutor_app"
    user_id = "demo_student"
    session_id = f"demo_session_{uuid.uuid4().hex[:8]}"
    session = await session_service.create_session(app_name=app_name, user_id=user_id, session_id=session_id)

    runner = Runner(agent=tutor_agent, app_name=app_name, session_service=session_service)
    test_content = types.Content(parts=[types.Part(text=user_query)])

    print(f"\n👩‍🎓 Student: {user_query}")
    print(f"\n🎓 Tutor Agent response:")
    print("-" * 60)

    async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=test_content):
        if event.is_final_response() and event.content:
            for part in event.content.parts:
                if hasattr(part, "text"):
                    print(part.text)
    print("-" * 60)

In [57]:
# 11. Test Tutor Agent

print("🧪 Testing Educator/Tutor A2A Communication...\n")
print("Subject: Python Coding")
await test_a2a_communication("Can you explain Python loops?")

INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False
INFO:google_adk.google.adk.models.google_llm:Response received from the model.
INFO:google_adk.google.adk.agents.remote_a2a_agent:Successfully resolved remote A2A agent: knowledge_agent


In [58]:
# Test for Physics question
print("Subject: Physics")
await test_a2a_communication("Tell me about Newton's laws of motion.")

INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False
INFO:google_adk.google.adk.models.google_llm:Response received from the model.
INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False
INFO:google_adk.google.adk.models.google_llm:Response received from the model.


In [60]:
# Test for Math question
print("Subject: Math")
await test_a2a_communication("What is Matrix in Linear Algebra?")

INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False
INFO:google_adk.google.adk.models.google_llm:Response received from the model.
INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False
INFO:google_adk.google.adk.models.google_llm:Response received from the model.


In [61]:
# 12. Observability: Logging & Metrics

import logging
import time

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

async def logged_test(user_query: str):
    logging.info(f"User query: {user_query}")
    start_time = time.time()
    
    await test_a2a_communication(user_query)
    
    elapsed = time.time() - start_time
    logging.info(f"Response time: {elapsed:.2f}s")


In [62]:
# 13. Multi-Agent Extensions: Parallel Agents
# Set up multiple knowledge agents for different subjects (Math, Science, History) and have the Tutor Agent call them in parallel.

from asyncio import gather

async def parallel_subject_queries(queries):
    # Run multiple test queries at once
    await gather(*(test_a2a_communication(q) for q in queries))

# Example usage
subject_queries = [
    "Explain Newton's law.",
    "What's the chemical formula for water?",
    "Give World War II information?"
]
await parallel_subject_queries(subject_queries)

INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False
INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False
INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False
INFO:google_adk.google.adk.models.google_llm:Response received from the model.
INFO:google_adk.google.adk.models.google_llm:Response received from the model.
INFO:google_adk.google.adk.models.google_llm:Response received from the model.
INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False
INFO:google_adk.google.adk.models.google_llm:Response received from the model.


In [63]:
# 14. Agent Evaluation

expected_answers = {
    "Explain Newton's first law.": "An object at rest stays at rest...",
    "What's the chemical formula for water?": "H2O",
}

async def evaluate_response(user_query, expected_answer):
    # Capture the response
    from io import StringIO
    import sys

    buffer = StringIO()
    sys.stdout = buffer
    await test_a2a_communication(user_query)
    sys.stdout = sys.__stdout__
    
    response_text = buffer.getvalue()
    score = 1 if expected_answer.lower() in response_text.lower() else 0
    print(f"Query: {user_query}\nScore: {score}/1\n")
    
# Test
await evaluate_response("What's the chemical formula for water?", "H2O")


INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False
INFO:google_adk.google.adk.models.google_llm:Response received from the model.


In [64]:
# 15. Session & Memory Logging

session_history = []

async def session_logger(user_query):
    session_history.append({"query": user_query, "timestamp": time.time()})
    await test_a2a_communication(user_query)

# Example
await session_logger("Explain photosynthesis.")
print(session_history)


INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False
INFO:google_adk.google.adk.models.google_llm:Response received from the model.
